In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

# Vehicles

In [ ]:
num_vehicles = pd.Series([194348815, 59465369], ['Short WB', 'Long WB'])
avg_miles_traveled = pd.Series([11599, 11263], ['Short WB', 'Long WB'])
mpg = pd.Series([24.1, 17.6], ['Short WB', 'Long WB'])

print('Federal transportation stats have vehicle miles/mpg stats by short- and long-wheelbase.')
print('Some of the Short WB cars are actually EVs, but these will be ignored because they''re on only about 0.5% of the total.')
pd.DataFrame({
    'Millions of Vehicles': (num_vehicles / 1e6).round(1),
    'Avg. Miles Traveled/Year': avg_miles_traveled,
    'MPG': mpg
})

# https://www.fhwa.dot.gov/policyinformation/statistics/2019/pdf/vm1.pdf

In [ ]:
nvs, nvl = num_vehicles / num_vehicles.sum()
suv = 0.12
# https://hedgescompany.com/automotive-market-research-statistics/auto-mailing-lists-and-marketing/

frac_vehicles = pd.Series(
    [nvs, suv, nvl - suv],
    ['Car', 'SUV', 'Truck'])

num_vehicles3 = frac_vehicles * num_vehicles.sum()

ams, aml = avg_miles_traveled
avg_miles_traveled3 = pd.Series(
    [ams, aml, aml],
    ['Car', 'SUV', 'Truck'])

print('For EV calcs, we can break up SUVs and trucks if we assume 12% of vehicles are SUVs.')
print('This should help get a more realistic estimate of EV wh/mi for a replacement fleet.')
pd.DataFrame({
    'Millions of Vehicles': (num_vehicles3 / 1e6).round(1),
    'Avg. Miles Traveled/Year': avg_miles_traveled3
})

# ICE

In [ ]:
# refinery_efficiency = 0.865
# # https://publications.anl.gov/anlpubs/2011/01/69026.pdf

In [ ]:
refinery_yield = pd.Series(
    [3.5, 46.2, 0.1, 10.5, 0.1, 29.7, 2.1],
    ['Hydrocarbon Gas Liquids', 'Finished Motor Gasoline', 'Aviation Gasoline', 'Kerosene-Type Jet Fuel', 'Kerosene', 'Distillate Fuel Oil', 'Residual Fuel Oil'])
refinery_yield /= 100
# https://www.eia.gov/dnav/pet/pet_pnp_pct_dc_nus_pct_m.htm

print('Refineries produce multiple types of fuels from a barrel of oil.')
refinery_yield

In [ ]:
ice_gasoline_gal = (num_vehicles * avg_miles_traveled / mpg).sum()
gallon_barrel = 42
ice_oil_barrels = ice_gasoline_gal / refinery_yield['Finished Motor Gasoline'] / gallon_barrel
print(f'{ice_oil_barrels / 1e9:.1f} BBO refined to produce gasoline for ICE propulsion.')
print('More is consumed for other purposes.')

# EV

In [ ]:
transmission_losses = 0.05
# https://www.eia.gov/tools/faqs/faq.php?id=105&t=3

ev_other = 0.2
# https://www.energy.gov/eere/vehicles/articles/fotw-1045-september-3-2018-77-82-energy-put-electric-car-used-move-car-down

ev_efficiency_wh_mi = pd.Series(
    [280, 302, 98 * 1000 / 230],
    ['Car', 'SUV', 'Truck']
)
# https://ev-database.org/#sort:path~type~order=.rank~number~desc|make-checkbox-dropdown:pathGroup=.ford|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=9
# https://www.ford.com/trucks/f150/f150-lightning/2022

pd.DataFrame({
    'EV wh/mi': ev_efficiency_wh_mi.round()
})

# Generation

In [ ]:
ev_drive_wh = (num_vehicles3 * avg_miles_traveled3 * ev_efficiency_wh_mi).sum()
ev_drive_kwh = ev_drive_wh / 1000
print(f'EV driving demand requires {ev_drive_kwh / 1e9:.1f} TWh electrical.')

ev_gen_kwh = ev_drive_kwh / (1 - transmission_losses) / (1 - ev_other)
print(f'EV driving demand requires {ev_gen_kwh / 1e9:.1f} TWh thermal.')

In [ ]:
liter_gallon = 3.78541178
barrel_m3 = gallon_barrel * liter_gallon / 1000

oil_density_kg_m3 = (816 + 840) / 2 # midpoint of light oil
# https://www.cmegroup.com/content/dam/cmegroup/rulebook/NYMEX/2/200.pdf

barrel_kg = barrel_m3 * oil_density_kg_m3

In [ ]:
efficiency_ccgt = 0.6
# https://www.ipieca.org/resources/energy-efficiency-solutions/power-and-heat-generation/combined-cycle-gas-turbines/

In [ ]:
print('If gasoline demand is cut significantly, refinery operators should be expected to adjust the mix of products output.')
print('Strip out ICE demand for gasoline from refined product demand.')
other_refinery_demand = ice_oil_barrels * refinery_yield
other_refinery_demand['Finished Motor Gasoline'] = 0

In [ ]:
lhv_kwh_kg = pd.Series(
    [
        (12.58 + 13.28 + 12.88) / 3, # average of butane, ethane, propane
        12.06, 12.06, 11.94, 11.94,
        (11.83 + 11.28) / 2, # average of diesel and light fuel oil
        10.83
    ], 
    ['Hydrocarbon Gas Liquids', 'Finished Motor Gasoline', 'Aviation Gasoline', 'Kerosene-Type Jet Fuel', 'Kerosene', 'Distillate Fuel Oil', 'Residual Fuel Oil'])
# https://www.engineeringtoolbox.com/amp/fuels-higher-calorific-values-d_169.html

In [ ]:
c = np.ones(14)

# set minimum on the fraction yielded of each product
min_frac = refinery_yield / refinery_yield.sum() / 2
A_lb = np.zeros((7, 14))
for i in range(7):
    A_lb[i, :7] = -min_frac[i]
    A_lb[i, i] = 1 - min_frac[i]

A_ub = -A_lb
b_ub = np.zeros(7)

# the total thermal energy per barrel
A_eq = np.ones((1, 14))
A_eq[0, :7] = barrel_kg * lhv_kwh_kg
A_eq[0, 7:] = -A_eq[0, :7]
A_eq

b_eq = np.array([ev_gen_kwh / efficiency_ccgt])

bounds = [(b, None) for b in other_refinery_demand] + [(b, b) for b in other_refinery_demand]
res = opt.linprog(c, A_ub, b_ub, A_eq, b_eq, bounds, method='highs-ds')
assert res.success
x = res.x[:7]
x

In [ ]:
ev_oil_barrels = x.sum() / refinery_yield.sum()
alt_refinery_yield = pd.Series(x / ev_oil_barrels, refinery_yield.index)

pd.DataFrame({
    'Refinery Yield': (100 * refinery_yield).round(1),
    'Alt. Refinery Yield': (100 * alt_refinery_yield).round(1)
})

In [ ]:
fuel_barrels = ev_oil_barrels * alt_refinery_yield - other_refinery_demand
assert (fuel_barrels > -0.5).all()
alt_gen_kwh = (fuel_barrels * barrel_kg * lhv_kwh_kg).sum() * efficiency_ccgt
assert abs(alt_gen_kwh - ev_gen_kwh) < 1

In [ ]:
bbo = pd.DataFrame({
    'ICE': ice_oil_barrels,
    'EV': ev_oil_barrels
}, index=['BBO'])
bbo['Diff'] = bbo['EV'] - bbo['ICE']
(bbo / 1e9).round(2)

In [ ]:
other_inputs = 0.05
print('Refineries obtain their energy from the oil input, but they also take in external energy, particularly natural gas.')
print('Refining less oil saves that external natural gas consumption.')

# Check

In [ ]:
gasoline_yield = 0.45
thermal_eff_ice = 0.2
eff_ev = 0.9
eff_transmission = 0.95
eff_ccgt = 0.6
eff_refining = 0.92

In [ ]:
gasoline_barrels = ice_oil_barrels * gasoline_yield
ev_barrels = gasoline_barrels * thermal_eff_ice / (eff_ev * eff_transmission * efficiency_ccgt)
check = (gasoline_barrels - ev_barrels) / eff_refining
print(f'As a check: save about {check / 1e9:.1f} BBO.')